In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler

In [4]:
df = pd.read_csv("diabetes.csv")

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [5]:
X = df[df.columns[:-1]].values
Y = df[df.columns[-1]].values

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
data = np.hstack((X, np.reshape(Y, (-1, 1))))
transformed_df = pd.DataFrame(data, columns=df.columns)

In [7]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [16]:
len(transformed_df[transformed_df["Outcome"]==1]), len(transformed_df[transformed_df["Outcome"]==0])

(500, 500)

In [15]:
over = RandomOverSampler()
X, Y = over.fit_resample(X, Y)
data = np.hstack((X, np.reshape(Y, (-1, 1))))
transformed_df = pd.DataFrame(data, columns=df.columns)

In [17]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=0)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=0)

In [24]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(16, activation='relu'), # if x <= 0 --> 0, x > 0 --> x
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])


In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [26]:
model.evaluate(X_train, Y_train)

19/19 [==============================] - 0s 2ms/step - loss: 0.6908 - accuracy: 0.5467


[0.690812885761261, 0.54666668176651]

In [27]:
model.evaluate(X_valid, Y_valid)

7/7 [==============================] - 0s 2ms/step - loss: 0.6896 - accuracy: 0.5600


[0.6896452903747559, 0.5600000023841858]

In [28]:
model.fit(X_train, Y_train, batch_size=16, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20
38/38 [==============================] - 1s 4ms/step - loss: 0.6737 - accuracy: 0.6067 - val_loss: 0.6585 - val_accuracy: 0.6550
Epoch 2/20
38/38 [==============================] - 0s 3ms/step - loss: 0.6363 - accuracy: 0.7050 - val_loss: 0.6237 - val_accuracy: 0.7200
Epoch 3/20
38/38 [==============================] - 0s 3ms/step - loss: 0.5946 - accuracy: 0.7267 - val_loss: 0.5790 - val_accuracy: 0.7400
Epoch 4/20
38/38 [==============================] - 0s 3ms/step - loss: 0.5501 - accuracy: 0.7400 - val_loss: 0.5431 - val_accuracy: 0.7450
Epoch 5/20
38/38 [==============================] - 0s 4ms/step - loss: 0.5227 - accuracy: 0.7550 - val_loss: 0.5229 - val_accuracy: 0.7200
Epoch 6/20
38/38 [==============================] - 0s 3ms/step - loss: 0.5079 - accuracy: 0.7550 - val_loss: 0.5071 - val_accuracy: 0.7350
Epoch 7/20
38/38 [==============================] - 0s 3ms/step - loss: 0.4942 - accuracy: 0.7633 - val_loss: 0.4978 - val_accuracy: 0.7450
Epoch 8/20
38/38 [==

In [29]:
model.evaluate(X_test, Y_test)

7/7 [==============================] - 0s 2ms/step - loss: 0.5053 - accuracy: 0.7650


[0.5052649974822998, 0.7649999856948853]